In [ ]:
pip install -U scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 56.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
try:
    from google.colab import drive

    # Google Drive'ı Colab ortamına bağla
    drive.mount('/gdrive/', force_remount=True)
    print("Google Drive bağlantısı başarıyla oluşturuldu.")
except Exception as e:
    print(f"Hata: {str(e)}")



Mounted at /gdrive/
Google Drive bağlantısı başarıyla oluşturuldu.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os

In [ ]:
print(os.listdir("/gdrive/My Drive/Colab Notebooks/Pneumonia/"))

['X-Ray']


In [ ]:
DATADIR = '/gdrive/My Drive/Colab Notebooks/Pneumonia/X-Ray'

CATEGORIES = CATEGORIES = ["Bacterial","Viral"]

In [ ]:
IMG_SIZE = 100

In [ ]:
import cv2
import os

all_data = []

def create_all_data():
    total_images = 0
    failed_images = 0

    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)

        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                if img_array is not None:
                    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    all_data.append([new_array, class_num])
                    total_images += 1
                else:
                    failed_images += 1
            except Exception as e:
                print(f"Error processing image {img} in category {category}: {str(e)}")
                failed_images += 1

    print("Toplam Görüntü Sayısı: ", total_images)
    print("Başarısız Görüntü Sayısı: ", failed_images)

create_all_data()


Toplam Görüntü Sayısı:  16167
Başarısız Görüntü Sayısı:  1


In [ ]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  16167


In [ ]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(11639, 100, 100, 3)
(3234, 100, 100, 3)
(1294, 100, 100, 3)
(11639,)
(3234,)
(1294,)


In [ ]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [ ]:
!pip install keras-tuner

In [23]:
# 1- Yapay Sinir Ağı Tasarla
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

# Convolutional layer 1
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 2
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 3
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten layer
model.add(Flatten())

# Dense (fully connected) layer 1
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))# Output layer
model.add(Dense(2, activation='softmax'))


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 2- Modelin Performansını Değerlendirme
history = model.fit(x_train, train_yCl, validation_data=(x_val, valid_yCl), epochs=25, batch_size=64)

# 3- Test verisinin değerlendirilmesi
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Tahminler
y_pred_prob = model.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1)


# One-hot encoding'i geri dönüştürme
y_test_labels = np.argmax(test_yCl, axis=1)

# Accuracy
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Accuracy: {accuracy}")

# Precision
precision = precision_score(y_test_labels, y_pred)
print(f"Precision: {precision}")

# Recall
recall = recall_score(y_test_labels, y_pred)
print(f"Recall: {recall}")

# F1 Score
f1 = f1_score(y_test_labels, y_pred)
print(f"F1 Score: {f1}")

# Karmaşıklık Matrisi
conf_matrix = confusion_matrix(y_test_labels, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
class_report = classification_report(y_test_labels, y_pred)
print("Classification Report:")
print(class_report)


Epoch 1/25
182/182 [==============================] - 14s 51ms/step - loss: 8.0570 - accuracy: 0.6576 - val_loss: 0.5525 - val_accuracy: 0.7535
Epoch 2/25
182/182 [==============================] - 9s 49ms/step - loss: 0.5550 - accuracy: 0.7316 - val_loss: 0.5422 - val_accuracy: 0.7318
Epoch 3/25
182/182 [==============================] - 8s 42ms/step - loss: 0.5372 - accuracy: 0.7389 - val_loss: 0.5102 - val_accuracy: 0.7736
Epoch 4/25
182/182 [==============================] - 8s 43ms/step - loss: 0.4959 - accuracy: 0.7621 - val_loss: 0.5156 - val_accuracy: 0.7674
Epoch 5/25
182/182 [==============================] - 8s 42ms/step - loss: 0.4690 - accuracy: 0.7800 - val_loss: 0.4684 - val_accuracy: 0.7844
Epoch 6/25
182/182 [==============================] - 8s 43ms/step - loss: 0.4553 - accuracy: 0.7870 - val_loss: 0.4474 - val_accuracy: 0.8022
Epoch 7/25
182/182 [==============================] - 8s 42ms/step - loss: 0.4140 - accuracy: 0.8102 - val_loss: 0.4050 - val_accuracy: 0.816